# SimulationAPI
This notebooks provides examples in how to use the lenstronomy.SimulationAPI modules in simulating (realistic) mock lenses taylored to a specific observation and instrument.

The module enables to use the astronomical magnitude conventions and can translate those into the lenstronomy core module configurations.

In [1]:
import copy
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

# lenstronomy module import
import lenstronomy.Util.data_util as data_util
import lenstronomy.Util.util as util
import lenstronomy.Plots.plot_util as plot_util
from lenstronomy.SimulationAPI.sim_api import SimAPI
from lenstronomy.LightModel.Profiles.gaussian import GaussianEllipse
gauss = GaussianEllipse()


## Define camera and observations
As an example, we define the camera and observational settings of a LSST-like observation. We define one camera setting and three different observations corresponding th g,r,i imaging.

For the complete list of possible settings, we refer to the SimulationAPI.observation_api classes. There are pre-configured settings which approximately mimic observations from current and future instruments. Be careful using those and check whether they are sufficiently accurate for your specific science case!

In [2]:
# Instrument setting from pre-defined configurations

from lenstronomy.SimulationAPI.ObservationConfig.LSST import LSST
LSST_g = LSST(band='g', psf_type='GAUSSIAN', coadd_years=10)
kwargs_g_band = LSST_g.kwargs_single_band()
LSST_r = LSST(band='r', psf_type='GAUSSIAN', coadd_years=10)
kwargs_r_band = LSST_r.kwargs_single_band()
LSST_i = LSST(band='i', psf_type='GAUSSIAN', coadd_years=10)
kwargs_i_band = LSST_i.kwargs_single_band()

# lenstronomy provides these setting to be imported with the SimulationAPI.observation_constructor routines.


In [3]:
#Tailored individual settings (as an example):


LSST_camera = {'read_noise': 10,  # std of noise generated by read-out (in units of electrons)
               'pixel_scale': 0.263,  # scale (in arcseonds) of pixels
               'ccd_gain': 4.5  # electrons/ADU (analog-to-digital unit). A gain of 8 means that the camera digitizes the CCD signal so that each ADU corresponds to 8 photoelectrons.
              }

LSST_g_band_obs = {'exposure_time': 90.,  # exposure time per image (in seconds)
                   'sky_brightness': 21.7,  # sky brightness (in magnitude per square arcseconds)
                   'magnitude_zero_point': 30,  # magnitude in which 1 count per second per arcsecond square is registered (in ADU's)
                   'num_exposures': 10,  # number of exposures that are combined
                   'seeing': 0.6,  # full width at half maximum of the PSF (if not specific psf_model is specified)
                   'psf_type': 'GAUSSIAN',  # string, type of PSF ('GAUSSIAN' and 'PIXEL' supported)
                   'kernel_point_source': None  # 2d numpy array, model of PSF centered with odd number of pixels per axis (optional when psf_type='PIXEL' is chosen)
                  }

LSST_r_band_obs = {'exposure_time': 90.,
                   'sky_brightness': 20.7,
                   'magnitude_zero_point': 30,
                   'num_exposures': 10,
                   'seeing': 0.6,
                   'psf_type': 'GAUSSIAN'}

LSST_i_band_obs = {'exposure_time': 90.,
                   'sky_brightness': 20.1,
                   'magnitude_zero_point': 30,
                   'num_exposures': 10,
                   'seeing': 0.6,
                   'psf_type': 'GAUSSIAN'}

# here we merge observations and instrument for the three bands
#kwargs_g_band = util.merge_dicts(LSST_camera, LSST_g_band_obs)
#kwargs_r_band = util.merge_dicts(LSST_camera, LSST_r_band_obs)
#kwargs_i_band = util.merge_dicts(LSST_camera, LSST_i_band_obs)

## Define model settings

The model settings are handled by the SimulationAPI.model_api ModelAPI class. 
The role is to return instances of the lenstronomy LightModel, LensModel, PointSource modules according to the options chosen by the user. Currently, all other model choices are equivalent to the ones provided by LightModel, LensModel, PointSource.
The current options of the class instance only describe a subset of possibilities and we refer to the specific class instances for details about all the possibilities.

For this example, we chose a single lens plane and a single source plane, elliptical Sersic profiles and an additional lensed point source.

In [4]:
kwargs_model = {'lens_model_list': ['SIE', 'SHEAR'],  # list of lens models to be used
                'lens_light_model_list': ['SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                'source_light_model_list': ['SERSIC_ELLIPSE'],  # list of extended source models to be used
                'point_source_model_list': ['SOURCE_POSITION']  # list of point source models to be used
    }

## Generate SimAPI instance for the different observations
Here we make an instanbce of the SimAPI class and execute the major tasks accessible as the interface to the ImSim core module.

In [5]:
numpix = 64  # number of pixels per axis of the image to be modelled

# here we define the numerical options used in the ImSim module. 
# Have a look at the ImageNumerics class for detailed descriptions.
# If not further specified, the default settings are used.
kwargs_numerics = {'point_source_supersampling_factor': 1}

sim_g = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model)
sim_r = SimAPI(numpix=numpix, kwargs_single_band=kwargs_r_band, kwargs_model=kwargs_model)
sim_i = SimAPI(numpix=numpix, kwargs_single_band=kwargs_i_band, kwargs_model=kwargs_model)

# return the ImSim instance. With this class instance, you can compute all the
# modelling accessible of the core modules. See class documentation and other notebooks.
imSim_g = sim_g.image_model_class(kwargs_numerics)
imSim_r = sim_r.image_model_class(kwargs_numerics)
imSim_i = sim_i.image_model_class(kwargs_numerics)

## Brightness definitions in magnitude space
One core feature is the support of light profile amplitudes in astronomical magnitude space (at least for few selected well defined brightness profiles).

We first define all parameters in magnitude space and then use the SimAPI routine to translate the arguments into lenstronomy conventions used by the ImSim module. The second model of each light component we defined as 'INTERPOL', which sets an interpolation grid given an image. This can be used to past real galaxies as lenses or sources into lenstronomy.

In [ ]:

# g-band

# lens light
kwargs_lens_light_mag_g = [{'magnitude': 14, 'R_sersic': .6, 'n_sersic': 4, 'e1': 0.1, 'e2': -0.1, 'center_x': 0, 'center_y': 0}]
# source light
kwargs_source_mag_g = [{'magnitude': 19, 'R_sersic': 0.3, 'n_sersic': 1, 'e1': -0.3, 'e2': -0.2, 'center_x': 0, 'center_y': 0}]
# point source
kwargs_ps_mag_g = [{'magnitude': 21, 'ra_source': 0.03, 'dec_source': 0}]



# and now we define the colors of the other two bands

# r-band
g_r_source = 1  # color mag_g - mag_r for source
g_r_lens = -1  # color mag_g - mag_r for lens light
g_r_ps = 0
kwargs_lens_light_mag_r = copy.deepcopy(kwargs_lens_light_mag_g)
kwargs_lens_light_mag_r[0]['magnitude'] -= g_r_lens

kwargs_source_mag_r = copy.deepcopy(kwargs_source_mag_g)
kwargs_source_mag_r[0]['magnitude'] -= g_r_source

kwargs_ps_mag_r = copy.deepcopy(kwargs_ps_mag_g)
kwargs_ps_mag_r[0]['magnitude'] -= g_r_ps


# i-band
g_i_source = 2
g_i_lens = -2
g_i_ps = 0
kwargs_lens_light_mag_i = copy.deepcopy(kwargs_lens_light_mag_g)
kwargs_lens_light_mag_i[0]['magnitude'] -= g_i_lens

kwargs_source_mag_i = copy.deepcopy(kwargs_source_mag_g)
kwargs_source_mag_i[0]['magnitude'] -= g_i_source

kwargs_ps_mag_i = copy.deepcopy(kwargs_ps_mag_g)
kwargs_ps_mag_i[0]['magnitude'] -= g_i_ps

# turn magnitude kwargs into lenstronomy kwargs
kwargs_lens_light_g, kwargs_source_g, kwargs_ps_g = sim_g.magnitude2amplitude(kwargs_lens_light_mag_g, kwargs_source_mag_g, kwargs_ps_mag_g)
kwargs_lens_light_r, kwargs_source_r, kwargs_ps_r = sim_r.magnitude2amplitude(kwargs_lens_light_mag_r, kwargs_source_mag_r, kwargs_ps_mag_r)
kwargs_lens_light_i, kwargs_source_i, kwargs_ps_i = sim_i.magnitude2amplitude(kwargs_lens_light_mag_i, kwargs_source_mag_i, kwargs_ps_mag_i)


## setting lens model parameters
And finally we need a lens model. The default lensing units are in observed deflection angle (arc seconds) mapping the image to the source plane (reduced angles). In a single lens and single source plane model, this is all we need to specify and no futher cosmology is required.

In [ ]:
kwargs_lens = [
    {'theta_E': 2., 'e1': 0.4, 'e2': -0.1, 'center_x': 0, 'center_y': 0},  # SIE model
    {'gamma1': 0.03, 'gamma2': 0.01, 'ra_0': 0, 'dec_0': 0}  # SHEAR model
]


## simluate image
Finally we can simulate the images with the ImageModel class instance and the lenstronomy parameters for the different bands. Note that in the specific example we included a point source (representing a quasar) in the center of the lensed galaxy. The SimulationAPI supports various options. Further down below we simulate multi-lens multi-source plane configurations too.

In [ ]:
image_g = imSim_g.image(kwargs_lens, kwargs_source_g, kwargs_lens_light_g, kwargs_ps_g)
image_r = imSim_r.image(kwargs_lens, kwargs_source_r, kwargs_lens_light_r, kwargs_ps_r)
image_i = imSim_i.image(kwargs_lens, kwargs_source_i, kwargs_lens_light_i, kwargs_ps_i)

# add noise
image_g += sim_g.noise_for_model(model=image_g)
image_r += sim_r.noise_for_model(model=image_r)
image_i += sim_i.noise_for_model(model=image_i)

# and plot it

img = np.zeros((image_g.shape[0], image_g.shape[1], 3), dtype=float)
img[:,:,0] = plot_util.sqrt(image_g, scale_min=0, scale_max=10000)
img[:,:,1] = plot_util.sqrt(image_r, scale_min=0, scale_max=10000)
img[:,:,2] = plot_util.sqrt(image_i, scale_min=0, scale_max=10000)

plt.clf()
plt.imshow(img, aspect='equal', origin='lower')
plt.show()

## creating mock images from postit-stamps of other images
One specific light model option is 'INTERPOL' where you can pass a 2d grid of surface brightness values (e.g. a real image). The 'INTERPOL' class then interpolates this grid and enables the surface brightness evaluation also under possible distortions. The 'INTERPOL' class also allows to scale the input image and rotate it to the desired angle. In the SimulationAPI, you can furthermore assigne the object a magnitude and the integrated surface brightness will be scaled to match the assigned total magnitude.

In the following example we create an 'image' with the surface brightness of the pixels representing a simple Gaussian distribution and then use different scales and magnitudes of this 'image' to represent the lensing galaxy and the lensed host galaxy of a point source.

In [ ]:
kwargs_model_postit = {'lens_model_list': ['SIE', 'SHEAR'],  # list of lens models to be used
                'lens_light_model_list': ['INTERPOL'],  # list of unlensed light models to be used
                'source_light_model_list': ['INTERPOL'],  # list of extended source models to be used
                'point_source_model_list': ['SOURCE_POSITION']  # list of point source models to be used
    }


sim_g = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model_postit)
sim_r = SimAPI(numpix=numpix, kwargs_single_band=kwargs_r_band, kwargs_model=kwargs_model_postit)
sim_i = SimAPI(numpix=numpix, kwargs_single_band=kwargs_i_band, kwargs_model=kwargs_model_postit)

# return the ImSim instance. With this class instance, you can compute all the
# modelling accessible of the core modules. See class documentation and other notebooks.
imSim_g = sim_g.image_model_class(kwargs_numerics)
imSim_r = sim_r.image_model_class(kwargs_numerics)
imSim_i = sim_i.image_model_class(kwargs_numerics)

# we make an 'image' based on a 2d gaussian
x_grid, y_grid = util.make_grid(numPix=10, deltapix=1)
flux = gauss.function(x_grid, y_grid, amp=1, sigma=2, e1=0.4, e2=0, center_x=0, center_y=0)
image_gauss = util.array2image(flux)

# g-band

# lens light
kwargs_lens_light_mag_g = [{'magnitude': 17, 'image': image_gauss, 'scale': 0.1, 'phi_G': 0, 'center_x': 0., 'center_y': 0}]
# source light
kwargs_source_mag_g = [{'magnitude': 22, 'image': image_gauss, 'scale': 0.03, 'phi_G': 0.4, 'center_x': 0., 'center_y': 0}]


# and now we define the colors of the other two bands

# r-band
g_r_source = 1  # color mag_g - mag_r for source
g_r_lens = -1  # color mag_g - mag_r for lens light
g_r_ps = 0
kwargs_lens_light_mag_r = copy.deepcopy(kwargs_lens_light_mag_g)
kwargs_lens_light_mag_r[0]['magnitude'] -= g_r_lens

kwargs_source_mag_r = copy.deepcopy(kwargs_source_mag_g)
kwargs_source_mag_r[0]['magnitude'] -= g_r_source

kwargs_ps_mag_r = copy.deepcopy(kwargs_ps_mag_g)
kwargs_ps_mag_r[0]['magnitude'] -= g_r_ps


# i-band
g_i_source = 2
g_i_lens = -2
g_i_ps = 0
kwargs_lens_light_mag_i = copy.deepcopy(kwargs_lens_light_mag_g)
kwargs_lens_light_mag_i[0]['magnitude'] -= g_i_lens

kwargs_source_mag_i = copy.deepcopy(kwargs_source_mag_g)
kwargs_source_mag_i[0]['magnitude'] -= g_i_source

kwargs_ps_mag_i = copy.deepcopy(kwargs_ps_mag_g)
kwargs_ps_mag_i[0]['magnitude'] -= g_i_ps

# turn magnitude kwargs into lenstronomy kwargs
kwargs_lens_light_g, kwargs_source_g, kwargs_ps_g = sim_g.magnitude2amplitude(kwargs_lens_light_mag_g, kwargs_source_mag_g, kwargs_ps_mag_g)
kwargs_lens_light_r, kwargs_source_r, kwargs_ps_r = sim_r.magnitude2amplitude(kwargs_lens_light_mag_r, kwargs_source_mag_r, kwargs_ps_mag_r)
kwargs_lens_light_i, kwargs_source_i, kwargs_ps_i = sim_i.magnitude2amplitude(kwargs_lens_light_mag_i, kwargs_source_mag_i, kwargs_ps_mag_i)

image_g = imSim_g.image(kwargs_lens, kwargs_source_g, kwargs_lens_light_g, kwargs_ps_g)
image_r = imSim_r.image(kwargs_lens, kwargs_source_r, kwargs_lens_light_r, kwargs_ps_r)
image_i = imSim_i.image(kwargs_lens, kwargs_source_i, kwargs_lens_light_i, kwargs_ps_i)

# add noise
image_g += sim_g.noise_for_model(model=image_g)
image_r += sim_r.noise_for_model(model=image_r)
image_i += sim_i.noise_for_model(model=image_i)

# and plot it

img = np.zeros((image_g.shape[0], image_g.shape[1], 3), dtype=float)
img[:,:,0] = plot_util.sqrt(image_g, scale_min=0, scale_max=1000)
img[:,:,1] = plot_util.sqrt(image_r, scale_min=0, scale_max=1000)
img[:,:,2] = plot_util.sqrt(image_i, scale_min=0, scale_max=1000)

plt.clf()
plt.imshow(img, aspect='equal', origin='lower')
plt.show()

## time variable source modeling
The SimulationAPI also comes with a class $\texttt{PointSourceVariability}$ which enables to model a variable (lensed) point source. It builts upon the base class $\texttt{SimAPI}$ and allows to provide a time-variable function to be passed and the simulation be made at different time steps. This can e.g. be used to simulate lensed supernovae and provides an example of how potentially even more complex time variablility simulations may be performed with lenstronomy.

In [ ]:
# we define a time variable function in magnitude space
def var_func(time):
    sigma = 5
    mag_0 = 19
    cps = np.exp(-time ** 2 / (2 * sigma ** 2))
    mag = data_util.cps2magnitude(cps, magnitude_zero_point=0)
    mag_norm = data_util.cps2magnitude(1, magnitude_zero_point=0)
    mag_return = mag - mag_norm + mag_0
    return mag_return

kwargs_model_time_var = {'lens_model_list': ['SIE', 'SHEAR'],  # list of lens models to be used
                'lens_light_model_list': ['SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                'source_light_model_list': ['SERSIC_ELLIPSE'],  # list of extended source models to be used
                         'z_lens': 0.5, 'z_source': 2
               }
# source position
source_x, source_y = 0.01, 0.05
# lens light
kwargs_lens_light_mag_g = [{'magnitude': 19, 'R_sersic': .6, 'n_sersic': 4, 'e1': 0.1, 'e2': -0.1, 'center_x': 0, 'center_y': 0}]
# source light
kwargs_source_mag_g = [{'magnitude': 22, 'R_sersic': 0.3, 'n_sersic': 1, 'e1': -0.3, 'e2': -0.2, 'center_x': 0, 'center_y': 0}]



from lenstronomy.SimulationAPI.point_source_variability import PointSourceVariability
ps_var = PointSourceVariability(source_x, source_y, var_func, numpix, kwargs_g_band, kwargs_model_time_var, kwargs_numerics,
                 kwargs_lens, kwargs_source_mag_g, kwargs_lens_light_mag_g, kwargs_ps_mag=None)

# these are the time delays of the multiple images (relative to the first appearing)
print(ps_var.delays)

In [ ]:
time = 10  # chose a time stamp of the image
image_g = ps_var.image_time(time=time)
image_g += sim_g.noise_for_model(model=image_g)
plt.matshow(np.log10(image_g), origin='lower')
plt.show()



## multi lens and multi source plane modeling (compound lenses)
Lenstronomy has full multi-plane ray-tracing capabilities and its design allows to place different sources (surface brightness models) at different redshifts (source planes). This functionality is also enable within the SimulationAPI module.

Multiple lens or source planes require the knowledge of the relative optical distances, or given their redshifts a specific cosmological model. The light models are defined relative to the angle to the specific redshift assuming a homogeneous line of sight (angular diameter distances of the background cosmology). The lensing deflection angles are defined in reduced deflections in respect of the lensing redshift to a specified source redshift.


In [ ]:
# we define a specific cosmology
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.)

kwargs_model_composite = {'lens_model_list': ['SIE', 'SHEAR'],  # list of lens models to be used
                          'lens_redshift_list': [0.5, 0.5],  # list of redshift of the deflections
                          'lens_light_model_list': ['SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                          'source_light_model_list': ['SERSIC_ELLIPSE', 'SERSIC'],  # list of extended source models to be used
                          'source_redshift_list': [0.8, 2.5],  # list of redshfits of the sources in same order as source_light_model_list
                          'cosmo': cosmo,  # astropy.cosmology instance
                          'z_source_convention': 3,  # source redshfit to which the reduced deflections are computed, is the maximal redshift of the ray-tracing
                          'z_source': 3,  # redshift of the default source (if not further specified by 'source_redshift_list') and also serves as the redshift of lensed point sources
    }

# we only simulate here the g-band, see above how to combine multiple bands
sim_g = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model_composite)
imSim_g = sim_g.image_model_class(kwargs_numerics)

kwargs_lens = [
    {'theta_E': 2., 'e1': 0.1, 'e2': -0.1, 'center_x': 0, 'center_y': 0},  # SIE model
    {'gamma1': 0.03, 'gamma2': 0.01}  # SHEAR model
]
# lens light
kwargs_lens_light_mag_g = [{'magnitude': 20, 'R_sersic': .6, 'n_sersic': 4, 'e1': 0.1, 'e2': -0.1, 'center_x': 0, 'center_y': 0}]
# source light
kwargs_source_mag_g = [{'magnitude': 22, 'R_sersic': 0.3, 'n_sersic': 1, 'e1': -0.3, 'e2': -0.2, 'center_x': 0, 'center_y': 0},
                      {'magnitude': 23, 'R_sersic': 0.8, 'n_sersic': 2, 'center_x': 0.2, 'center_y': 0}]
kwargs_lens_light_g, kwargs_source_g, kwargs_ps_g = sim_g.magnitude2amplitude(kwargs_lens_light_mag_g, kwargs_source_mag_g, kwargs_ps_mag_g)


image_g = imSim_g.image(kwargs_lens, kwargs_source_g, kwargs_lens_light_g)
image_g += sim_g.noise_for_model(model=image_g)

plt.matshow(np.log10(image_g))
plt.show()

## Using physical deflector units
The default setting for the lens model parameters are reduced deflector units. The $\texttt{SimulationAPI}$ module provides a convenient function to use physical mass unit parameterizations and to wrapp them into the standard reduced deflection angle convention of $\texttt{lenstronomy}$.

For lens models parameterized with Einstein radius $\theta_E$, the velocity dispersion 'sigma_v' can be set instead (in units km/s) and given a redshift of lens and source and a specified cosmology, the Einstein radius is computed. For NFW profiles, 'M200' and 'concentration' can be set instead of 'alpha_Rs' and 'Rs' and those will be converted into the referenced lensing quantities.

In [ ]:
kwargs_model_physical = {'lens_model_list': ['SIE', 'NFW'],  # list of lens models to be used
                          'lens_redshift_list': [0.5, 0.3],  # list of redshift of the deflections
                          'lens_light_model_list': ['SERSIC_ELLIPSE'],  # list of unlensed light models to be used
                          'source_light_model_list': ['SERSIC_ELLIPSE', 'SERSIC'],  # list of extended source models to be used
                          'source_redshift_list': [0.8, 2.5],  # list of redshfits of the sources in same order as source_light_model_list
                          'cosmo': cosmo,  # astropy.cosmology instance
                          'z_source_convention': 3,  # source redshfit to which the reduced deflections are computed, is the maximal redshift of the ray-tracing
                          'z_source': 3,  # redshift of the default source (if not further specified by 'source_redshift_list') and also serves as the redshift of lensed point sources
    }

kwargs_mass = [{'sigma_v': 250, 'center_x': 0, 'center_y': 0, 'e1': 0.1, 'e2': 0},
              {'M200': 10**13, 'concentration': 4, 'center_x': 1, 'center_y': 0}]

sim = SimAPI(numpix=numpix, kwargs_single_band=kwargs_g_band, kwargs_model=kwargs_model_physical)

imSim = sim.image_model_class(kwargs_numerics)

kwargs_lens = sim.physical2lensing_conversion(kwargs_mass=kwargs_mass)
print(kwargs_lens)

# lens light
kwargs_lens_light_mag_g = [{'magnitude': 20, 'R_sersic': .6, 'n_sersic': 4, 'e1': 0.1, 'e2': -0.1, 'center_x': 0, 'center_y': 0}]
# source light
kwargs_source_mag_g = [{'magnitude': 22, 'R_sersic': 0.3, 'n_sersic': 1, 'e1': -0.3, 'e2': -0.2, 'center_x': 0, 'center_y': 0},
                      {'magnitude': 21, 'R_sersic': 0.8, 'n_sersic': 2, 'center_x': 0.2, 'center_y': 0}]
kwargs_lens_light_g, kwargs_source_g, kwargs_ps_g = sim_g.magnitude2amplitude(kwargs_lens_light_mag_g, kwargs_source_mag_g, kwargs_ps_mag_g)


image_g = imSim.image(kwargs_lens, kwargs_source_g, kwargs_lens_light_g)
image_g += sim.noise_for_model(model=image_g)

plt.matshow(np.log10(image_g))
plt.show()
